IMPORTING THE REQUIRED LIBRARIES

In [4]:

import pandas as pd
import numpy as np
import random
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
%matplotlib inline

IMPORTING THE REQUIRED DATASETS

In [6]:

aisles = pd.read_csv("dataset/aisles.csv")
departments = pd.read_csv("dataset/departments.csv")
order_products__prior = pd.read_csv("dataset/order_products__prior.csv")
order_products__train = pd.read_csv("dataset/order_products__train.csv")
orders = pd.read_csv("dataset/orders.csv")
products = pd.read_csv("dataset/products.csv")

MERGING THE DATASETS TO A SINGLE DATAFRAME

In [7]:
#merging on product_id,department_id,aisle_id to get information about products,departments,aisles
dataset=order_products__train.merge(products,how="left",on="product_id")
dataset=dataset.merge(departments,how="left",on="department_id")
dataset=dataset.merge(aisles,how="left",on="aisle_id")
dataset.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses
2,1,10246,3,0,Organic Celery Hearts,83,4,produce,fresh vegetables
3,1,49683,4,0,Cucumber Kirby,83,4,produce,fresh vegetables
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood


CONSIDERING ONLY THOSE ROWS IN WHICH THE PRODUCT IS RE-ORDERED

In [8]:

reorders=dataset[dataset.reordered==1]
#typecsting the product_id to int64 for firthur compution
reorders.product_id=reorders.product_id.astype('int64')
print(len(reorders.order_id))
reorders.head()

828824


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood
7,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese
9,36,19660,2,1,Spring Water,115,7,beverages,water seltzer sparkling water


CONSIDERING THE ROWS IN WHICH THE PRODUCT IS ORDERED MORE THAN ONES<br/>IF THE PRODUCT IS ORDERED MORE THAN ONES INDICATES <br/>THAT THE PRODUT WILL BE PICKED UP BY  USER MORE OFTEN

In [9]:
#creating filter to extract products that have been order more than ones in addition to being re-ordered
hivol = reorders.copy()['product_id'].value_counts().sort_values(ascending=False)\
    [reorders.copy()['product_id'].value_counts().sort_values(ascending=False) > 1].index.tolist()

In [11]:
# applying the above mask to the dataframe
reorders = reorders[reorders['product_id'].isin(hivol)]
print(len(reorders))
reorders.head()

820954


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood
7,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese
9,36,19660,2,1,Spring Water,115,7,beverages,water seltzer sparkling water


In [13]:
#creating a new row which indicates whether a product is ordered multiple times or not high demand
reorders['hi_dem'] = (reorders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)
reorders.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,hi_dem
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt,NaN
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,True
4,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood,True
7,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,NaN
9,36,19660,2,1,Spring Water,115,7,beverages,water seltzer sparkling water,True


RECOMMENDATION SYSTEM

In [14]:
#merging the data sets ->orders with order to get user_id of that perticular transaction
user_orders = reorders.merge(orders)
print(len(user_orders.order_id))
user_orders.head()


820954


,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,hi_dem,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt,NaN,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,True,112108,train,4,4,10,9.0
2,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood,True,112108,train,4,4,10,9.0
3,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,NaN,112108,train,4,4,10,9.0
4,36,19660,2,1,Spring Water,115,7,beverages,water seltzer sparkling water,True,79431,train,23,6,18,30.0


In [205]:
#creating a new row which indicates whether a product is ordered multiple times or not high demand
user_orders['hi_dem'] = (user_orders.copy()['product_id'].value_counts().sort_values(ascending=False)>1)
user_orders.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,hi_dem,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,Bulgarian Yogurt,120,16,dairy eggs,yogurt,NaN,112108,train,4,4,10,9.0
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,True,112108,train,4,4,10,9.0
2,1,43633,5,1,Lightly Smoked Sardines in Olive Oil,95,15,canned goods,canned meat seafood,NaN,112108,train,4,4,10,9.0
3,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,True,112108,train,4,4,10,9.0
4,36,19660,2,1,Spring Water,115,7,beverages,water seltzer sparkling water,True,79431,train,23,6,18,30.0


In [206]:
#considering only products that have been ordered multiple times
hidem_ord = user_orders[user_orders['hi_dem'] == True]
hidem_ord.head()

,order_id,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,department,aisle,hi_dem,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
1,1,11109,2,1,Organic 4% Milk Fat Whole Milk Cottage Cheese,108,16,dairy eggs,other creams cheeses,True,112108,train,4,4,10,9.0
3,1,22035,8,1,Organic Whole String Cheese,21,16,dairy eggs,packaged cheese,True,112108,train,4,4,10,9.0
4,36,19660,2,1,Spring Water,115,7,beverages,water seltzer sparkling water,True,79431,train,23,6,18,30.0
8,36,48679,7,1,Organic Garnet Sweet Potato (Yam),83,4,produce,fresh vegetables,True,79431,train,23,6,18,30.0
9,36,46979,8,1,Asparagus,83,4,produce,fresh vegetables,True,79431,train,23,6,18,30.0


In [207]:
#comparing users to other users
#grouping uers to their produts to get the count of number of times the product is being ordered by that user
users = hidem_ord.groupby(['user_id','product_name']).size().sort_values(ascending=False).unstack().fillna(0)

In [208]:
#generating user similarity matrix by using cosine similarity
users_sim = pd.DataFrame(cosine_similarity(users),index=users.index,columns=users.index)
users_sim.head()

user_id,55,56,66,79,110,215,223,249,283,285,310,325,371,382,393,444,489,499,504,586,600,604,605,617,638,649,685,751,756,757,786,816,846,852,857,882,933,944,984,990,...,204954,205080,205148,205212,205220,205296,205383,205406,205419,205421,205429,205468,205508,205584,205594,205635,205641,205650,205656,205708,205724,205739,205747,205873,205878,205927,205950,205958,205965,205972,206023,206040,206082,206111,206136,206140,206158,206162,206177,206186
user_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
55,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.166667,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
56,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.188982,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
66,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
79,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0
110,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.235702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.111111,0.0,0.136083,0.235702,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.117851,0.333333,0.0,0.0,0.0,0.0,0.117851,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.235702,0.0,0.0,0.235702,0.0,0.0,0.0,0.000000,0.0,0.0,0.235702,0.0,0.0,0.0,0.0


In [209]:
#function return similar users/Items 
def next_prod(df,num_col):
    #extracting the column with the given column and droping the dropping the the current entriy which will be always 1 in the 
    #similarity matrix and then sorting the vlaues to get the top values
    return df[df.columns[num_col]].drop(df.columns[num_col]).sort_values(ascending=False).head(3)

In [210]:
#testing for the  56th user not user_id=56
#output being the top 3 similar users to the given user
pd.DataFrame(next_prod(users_sim,56)).T

user_id,43254,48962,10453
1711,0.5,0.353553,0.188982


RECOMMENDING PRODUCTS BASED ON THE USER_ID

In [211]:
#grouping produts and user to generate produts similarity matrix
products = hidem_ord.groupby(['product_name','user_id']).size().sort_values(ascending=False).unstack().fillna(0)

In [212]:
#generating produts imilarity matrix using cosine similarity
products_sim = pd.DataFrame(cosine_similarity(products),index=products.index,columns=products.index)

In [214]:
#recommendingg products to user based on the last product added to the cart
pd.DataFrame(next_prod(products_sim,11)).T

product_name,Organic Heirloom Tomatoes,Organic Spinach Bunch,Salted Butter
1% Low Fat Milk,0.57735,0.160128,0.144338


In [215]:
profile = pd.Series(np.zeros(len(products.columns.tolist())),users_sim[55])
profile.head()

55
1.0    0.0
0.0    0.0
0.0    0.0
0.0    0.0
0.0    0.0
dtype: float64

In [216]:
#performing dot product between the produts matix and users similar to user number 55 to get the most frequently purched products
#by the other users who are similar to user number 55
recommendations = np.dot(products.values,users_sim[55])

In [225]:
#creating a dataframe to get the top frequently purchased product
recommendations = pd.Series(recommendations, index=products.index)

In [218]:
#getting the top products that are more likely to be bought by the user
recommendations.sort_values(ascending=False).head()

product_name
Grape White/Green Seedless        11.120541
Flour Tortillas                    1.902369
Chocolate Peppermint Stick Bar     1.673560
Banana                             1.399944
Organic Avocado                    0.740677
dtype: float64

GENERALISING THE ABOVE METHOD 

In [219]:
#function to recommend 
def Recommender_System(user_id):
    #grouping each user with their products to get whether the produt is ordered by the user or not is not fill it with 0
    u = hidem_ord.groupby(['user_id','product_name']).size().sort_values(ascending=False).unstack().fillna(0)
    #generating user-user similarity matrix based on the products purchased
    u_sim = pd.DataFrame(cosine_similarity(u),index=u.index,columns=u.index)
    
    #grouping each produts with their users to get the list of users that have ordered the product is not fill with 0
    p = hidem_ord.groupby(['product_name','user_id']).size().sort_values(ascending=False).unstack().fillna(0)
    
    #getting the produts that are most likely to be bought by the user based on user-user similarity 
    recommendations = pd.Series(np.dot(p.values,u_sim[user_id]), index=p.index)
    #returning the poducts with the highest score
    return recommendations.sort_values(ascending=False).head()

In [221]:
#testing on a random user.
random.sample(hidem_ord['user_id'].tolist(),1)

[184689]

In [222]:
Recommender_System(184689)

(product_name
 Organic Grape Tomatoes    25.982305
 Banana                     2.367790
 Organic Baby Spinach       2.194510
 Organic Strawberries       2.013922
 Fresh Cauliflower          1.616025
 dtype: float64,)